In [1]:
import numpy as np

In [2]:
class Node:
         
    def __init__(self, initial, parent=None, acao=None, custo=0):

        self.initial = initial
    
    def __repr__(self):
        return "<Node {}>".format(self.initial)

    def expandir(self, problema, rainha=None):

        if rainha == None:
            rainha_list = self.encontrar_rainha()
            sorteio = np.random.choice(range(len(rainha_list)))
            rainha = rainha_list[sorteio]
        
        return [self.child_node(problema, rainha, acao) 
                for acao in problema.pos_mov(self.initial)]
        

    def child_node(self, problem, rainha, acao):

        prox_estado = problem.jogada(self.initial, rainha, acao)

        prox_node = Node(prox_estado, self, acao, 0)

        return prox_node
    
    def encontrar_rainha(self):

        coordenadas = []
        fil = len(self.initial)
        col = len(self.initial[:1][0])
        for i in range(fil):
            for j in range(col):
                if self.initial[i][j] == 1:
                    coordenadas.append([i,j])
              
        return coordenadas

In [8]:
class oitorainhas:

    def __init__(self, x, y, tabuleiro=None):
        self.x = x
        self.y = y

        if tabuleiro == None:
            self.tabuleiro = np.zeros([x,y])
            rainhas = 0
            while True:

                xx =  np.random.randint(self.x)
                yy =  np.random.randint(self.y)

                if self.tabuleiro[xx][yy] != 1:
                    self.tabuleiro[xx][yy] = 1
                    rainhas+=1
                    
                if rainhas ==  self.x:
                    break
                

    def pos_mov(self, tabuleiro):

        pos = []
        for i in range(self.x):
            for j in range(self.y):
                if tabuleiro[i][j] != 1:
                    pos.append([i,j])
        
        return pos

    def jogada(self, tabuleiro, rainha, coordenada):

        tab = tabuleiro.copy()

        tab[rainha[0],rainha[1]] = 0
        tab[coordenada[0],coordenada[1]] = 1

        return tab
    
    def test_objetivo(self, tabuleiro):

        invalido = 0

        for icol in range(self.y):

            for irow in range(self.x):
    
    
                if(tabuleiro[irow][icol] == 1):
               

                    # Contar 1s na coluna icol
                    for k in range(self.y):
                        if (tabuleiro[irow][k] == 1 and [irow, icol]!=[irow, k]):
                            invalido+=1

                    # Contar 1s na fileira irow
                    for k in range(self.x):
                        if (tabuleiro[k][icol] == 1 and [irow, icol]!=[k, icol]):
                            invalido+=1

                    # encontrar diagonal esquerda (secundaria)
                    col_esq = icol
                    row_esq = irow
                    while(col_esq != 0 and row_esq != (self.x-1)):
                        col_esq-=1
                        row_esq+=1       

                    # cencontrar diagonal direita (primaria)
                    col_dir = icol
                    row_dir = irow
                    while(col_dir != (self.y-1) and row_dir != (self.x-1)):
                        col_dir+=1
                        row_dir+=1

                    # contar 1's diagonal esquerda (secundaria)
                    while(row_esq >= 0 and col_esq <= self.y-1):

                        if (tabuleiro[row_esq][col_esq] == 1 and [irow, icol]!=[row_esq, col_esq]):
                            invalido+=1

                        col_esq+=1
                        row_esq-=1

                    # contar 1's diagonal direita (primaria)
                    while(col_dir>=0 and row_dir>=0 and [irow, icol]!=[row_dir, col_dir]):

                        if (tabuleiro[row_dir][col_dir] == 1):
                            invalido+=1
                            
                        col_dir-=1
                        row_dir-=1
       
        if invalido == 0:
            return True
        else:
            return False

Será feito um algoritmo de busca em largura (utilizado anteriormente para buscar caminhos) que irá buscar resolver o desafio das rainhas. Esse tipo de algoritmo para esse tipo de problema é ruim, já que o desafio não pede uma descrição de rotas podemos utilizar soluções que não se preocupam com isso e portanto utilizam menos memória e por não gerenciar muitos estados podem progredir mais rapidamente.

In [33]:
def Busca_largura(prob):

    # Lista que irá manter os estados 
    frontier = [Node(prob.tabuleiro)]

    # Verificar se o primeiro estado é o objetivo
    if prob.test_objetivo(frontier[0].initial):
            return frontier[0]
    
    while frontier:

        '''Selecionar o sempre o primeiro estado
        da lista, ou seja, sempre os mais rasos''' 
        no = frontier[0]

        # Novos estados derivados do nó selecionado
        filhos = no.expandir(prob)

        # Loop por cada um dos estados recém criados
        for filho in filhos:

            # Flag identificando se não é um estado repetido
            unico = False
            
            '''Irá acontecer um loop por toda a lista de
            estados verificando se os que foram recém criados
            já estão na lista'''
            for i in range(len(frontier)):

                if filho != frontier[i]:
                    unico = True

            # Caso o estado não exista na lista
            if unico:
                # Será feito o teste se é o objetivo
                if prob.test_objetivo(filho.initial):

                    return filho
                # Será adicionado à lista
                frontier.append(filho)

        # Iremos remover o estado que gerou a expansão 
        frontier.remove(no)

Um ambiente até 5x5 é computável para esse algoritmo de busca em largura, mais do que isso o tempo passa a ser grande

In [35]:
desafio = oitorainhas(5,5)
desafio.tabuleiro

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1.]])

In [36]:
solucao = Busca_largura(desafio)
solucao.initial

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])